## Question 1

In [307]:
def special_sum(lst):
    return [sum(lst)-i for i in lst]

In [308]:
special_sum([1,2,3,4])

[9, 8, 7, 6]

## Question 2

In [309]:
import numpy as np
import pandas as pd

## 2(a)

In [310]:
dt=pd.read_csv('Sales_Transactions_Dataset_Weekly.csv')

## 2(b)

In [312]:
dtt=dt.iloc[:,0:53]

dtt['median']=dtt.median(axis=1)
dtt['mean']=dtt.mean(axis=1)
dtt['min']=dtt.min(axis=1)
dtt['max']=dtt.max(axis=1)
dtt.iloc[:,53:].head()

,median,mean,min,max
0,10.0,9.641509,3.0,21.0
1,3.5,3.971698,0.0,10.0
2,8.0,8.679245,3.0,14.0
3,8.0,8.264151,2.0,19.0
4,8.0,8.452830,3.0,18.0


## 2(c)

In [313]:
dtt['sum']=dtt.sum(axis=1)
dtt.sort_values(by='sum', ascending=False).iloc[0,0]

'P409'

## 2(d)

In [314]:
def trendUp(dta):
    coeffs = np.polyfit(np.array(range(dta.count())), list(dta), 1)
    slope = coeffs[-2]
    return slope

In [315]:
a=dtt.iloc[0,1:]
dtt['trendUp']=dtt.iloc[:,1:53].apply(trendUp,axis=1)
dtt.sort_values(by='trendUp', ascending=False).iloc[0,0]

'P674'

## 2(e)

In [316]:
b=0
for i in range(0,52,2):
    dtt["2W_"+str(b)]=dtt['W'+str(i)]+dtt['W'+str(i+1)]
    b+=1

In [317]:
pd.concat([pd.DataFrame([['2W_'+str(i)]+dtt.sort_values(by='2W_'+str(i), ascending=True).iloc[:5,0].tolist()],
                        columns=['BiWeekly','Worst','SecWorst','ThiWorrs','ForWorst','FifWorst']) for i in range(int(52/2))],
          ignore_index=True).head()

,BiWeekly,Worst,SecWorst,ThiWorrs,ForWorst,FifWorst
0,2W_0,P381,P685,P609,P684,P290
1,2W_1,P819,P690,P287,P283,P282
2,2W_2,P819,P258,P256,P253,P252
3,2W_3,P819,P274,P273,P272,P668
4,2W_4,P819,P465,P258,P697,P463


## 2(f)(g)

In [318]:
def get_outliers(sales_data):
    #print('Outliers by week \n')
    sales_np_array = np.array(sales_data.loc[:, sales_data.columns[1:53]])
    mn=[np.mean(row) for row in sales_np_array]
    sd=[np.std(row) for row in sales_np_array]
    z=[[(x-mn[i])/sd[i] for x in sales_np_array[i]] for i in range(len(sales_np_array.tolist()))]
    outlier=[[sales_data.iloc[r,0],[sales_data.columns[l+1] for l in range(len(z[r])) if z[r][l]>3]] for r in range(len(z)) if len([k for k in z[r] if k>3])>0]
    return pd.DataFrame(outlier, columns=['Product_code','Week with Outlier'])

In [319]:
get_outliers(dt).head()

,Product_code,Week with Outlier
0,P1,[W7]
1,P4,[W35]
2,P14,[W14]
3,P29,[W17]
4,P36,[W12]


## Question 3

## 3(a)

In [320]:
data=pd.read_csv('data job posts.csv')

In [321]:
data.columns

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')

## 3(b)

In [322]:
datab=data[['Title','Duration','Location','JobDescription','JobRequirment','RequiredQual','Salary','Deadline','AboutC']]

## 3(c)

In [323]:
maxYear=data[['Year']].max(axis=0)[0]
data[data['Year']>maxYear-2].groupby('Company')['Title'] \
                                .count() \
                                .reset_index(name='count') \
                                .sort_values(['count'], ascending=False) \
                                .iloc[0,0]

'Mentor Graphics Development Services CJSC'

## 3(d)

In [324]:
data.groupby('Month')['Title'] \
    .count() \
    .reset_index(name='count') \
    .sort_values(['count'], ascending=False) \
    .iloc[0,0]

3

## 3 (e)

In [325]:
#pip install -U textblob
#nltk.download_shell()

In [326]:
import string
import nltk
#pip install -U textblob
#nltk.download_shell()
from textblob import Word
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
stop = stopwords.words('english')

In [327]:
def cleanDesc(desc):
    try:
        return ''.join([('' if c[0] in string.punctuation else ' ')+c for c in \
                    [Word(wrd).singularize() \
                     for wrd in word_tokenize(desc) if wrd.lower() not in stop \
                    ]]).strip()
    except:
        return desc

In [328]:
data['newJobDesc']=data['JobDescription'].apply(lambda x: cleanDesc(x))

In [329]:
data.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT,newJobDesc
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False,AMERIum Investment Consulting Company seeking ...
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False,NaN
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False,Public outreach strengthening growing network ...
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False,LEAD( Local Enhancement Development Health) BC...
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True,NaN


## 3(f)

In [330]:
data.Duration.unique()

array([nan, '3 months', 'Renewable annual contract\r\nPOSITION', ...,
       'Short-term, from 18 January 2016 to 15 April 2016.',
       'Long-term, with a probation period of 3 months.', '4 - 5 months'],
      dtype=object)

In [331]:
def nanHandler(dur):
    try:
        dur.split(' ')
        return dur
    except:
        return 'Not specified'

In [332]:
data['Duration']=data['Duration'].apply(lambda x: nanHandler(x))

In [333]:
data.Duration.head()

0                            Not specified
1                                 3 months
2    Renewable annual contract\r\nPOSITION
3                            Not specified
4                            Not specified
Name: Duration, dtype: object

## 3(g)

In [334]:
newData=data

## 3(h)

## Question 4

In [140]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Assesment').getOrCreate()

## 4(b)

In [174]:
df=spark.read.csv('test.csv',header=True)

In [143]:
df.printSchema()

root
 |-- test_id: string (nullable = true)
 |-- description_x: string (nullable = true)
 |-- description_y: string (nullable = true)
 |-- same_security: string (nullable = true)



In [145]:
df.show(5)

+-------+--------------------+--------------------+-------------+
|test_id|       description_x|       description_y|same_security|
+-------+--------------------+--------------------+-------------+
|      0|        semtech corp| semtech corporation|         null|
|      1|vanguard mid cap ...|vanguard midcap i...|         null|
|      2|spdr gold trust g...|spdr gold trust s...|         null|
|      3|vanguard total bo...|vanguard total bo...|         null|
|      4|oakmark internati...|oakmark internati...|         null|
+-------+--------------------+--------------------+-------------+
only showing top 5 rows



## 4(c)

In [288]:
import re
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

In [302]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]


def str_match_score(s1, s2):
    s1 = re.sub(r'\s+', ' ', s1).strip()
    s2 = re.sub(r'\s+', ' ', s2).strip()
    score = 0
    a1=set(s1.split(" "))
    b1=set(s2.split(" "))
    if len(a1) > len(b1):
        (a1, b1) = (b1, a1)
    for a in a1:
        max_score = 0
        for b in b1:
            max_score = max(max_score, 1 - levenshtein(a, b) / max(float(len(a)), float(len(b))))
        score += max_score
    score = score / len(b1)
    return float(score)

In [303]:
func =  F.udf(lambda x:str_match_score(x['s1'],x['s2']),FloatType())

In [304]:
df1 = df.withColumn('score', 
                   func(F.struct(F.col('description_x').alias('s1'),
                                 F.col('description_y').alias('s2')))
                  )

In [305]:
df1.printSchema()

root
 |-- test_id: string (nullable = true)
 |-- description_x: string (nullable = true)
 |-- description_y: string (nullable = true)
 |-- same_security: string (nullable = true)
 |-- score: float (nullable = true)



## 4(d)

In [295]:
from collections import Counter as mset 
def str_match_score_2(v1,v2):
    try :
        if (v1 == None or v2 == None):
                return 0
        v1b = v1.split(' ')
        v1b1 = []
        for bi in v1b:
                v1b1.extend([bi[i:i+2].replace(' ','') for i in range(0, len(bi), 2)])
        v2b = v2.split(' ')
        v2b1 = []
        for bi in v2b:
                v2b1.extend([bi[i:i+2].replace(' ','') for i in range(0, len(bi), 2)])
        if (len(v1b1) < len(v2b1)):
                scoreb = 1*(len(list((mset(v1b1) & mset(v2b1)).elements()))/len(v2b1))
        else:
                scoreb = 1*(len(list((mset(v1b1) & mset(v2b1)).elements()))/len(v1b1))
        return scoreb
    except :
        return 0

In [296]:
func= F.udf(lambda x:str_match_score_2(x['s1'],x['s2']),FloatType())

In [297]:
df2 = df.withColumn('Faster_score', 
                   func(F.struct(F.col('description_x').alias('s1'),
                                 F.col('description_y').alias('s2')))
                  )

In [300]:
df2.printSchema()

root
 |-- test_id: string (nullable = true)
 |-- description_x: string (nullable = true)
 |-- description_y: string (nullable = true)
 |-- same_security: string (nullable = true)
 |-- Faster_score: float (nullable = true)

